In [195]:
import numpy as np


In [219]:
def get_PQ(mat,answ):
    P1 = -mat[0,1]/mat[0,0]
    Q1 = answ[0,0]/mat[0,0]
    size_mat = mat[:,0].size
    i = 1
    P = [P1]
    Q = [Q1]
    while i < size_mat-1:
        P.append(-mat[i,i+1]/(mat[i,i]+mat[i,i-1]*P[i-1]))
        Q.append((answ[i,0]-mat[i,i-1]*Q[i-1])/(mat[i,i]+mat[i,i-1]*P[i-1]))
        i+=1
    P.append(0)
    Q.append((answ[i,0]-mat[i,i-1]*Q[i-1])/(mat[i,i]+mat[i,i-1]*P[i-1]))
    return P, Q

def method(size_mat, P, Q):
    X = np.zeros(size_mat)
    X[size_mat-1] = Q[size_mat-1]
    i = size_mat-2
    while i > -1:
        X[i] = X[i+1]*P[i]+Q[i]
        i-=1
    return X

def run_method(mat,answ):
    size_mat = mat[:,0].size
    answ = np.asarray(answ).reshape(-1,1)
    P, Q = get_PQ(mat,answ)
    X = method(size_mat, P, Q)
    return X

In [249]:
def h(i, x):
    return x[i]-x[i-1]

def getRow(i,x,f):
    n = len(x)
    A = np.zeros([n-2])

    if i == 0:
        A[0] = 2*(h(1, x)+h(2, x))
        A[1] = h(2, x)
        B = 3*(((f[2] - f[1]) / h(2, x)) - ((f[1] - f[0]) / h(1, x)))
    elif i == n-3:
        A[i-1] = h(n-2, x)
        A[i] = 2*(h(n-2, x)+h(n-1, x))
        B = 3*(((y[n-1] - y[n-2]) / h(n-1, x)) - ((y[n-2] - y[n-3]) / h(n-2, x)))
    elif i > 0:
        A[i-1] = h(i+1,x)
        A[i] = 2*(h(i+1, x)+h(i+2, x))
        A[i+1] = h(i+2,x)
        B = 3*(((y[i+2] - y[i+1]) / h(i+2, x)) - ((y[i+1] - y[i]) / h(i+1, x)))
    return A,B
    
def get_c(x, f):
    A = []
    B = []
    c = np.zeros(len(x)-1)
    for i in range(len(x)-2):
        a_tmp, b_tmp = getRow(i,x,f)
        A.append(a_tmp)
        B.append(b_tmp)
    c[1:] = run_method(np.array(A),np.array(B))
    return c

def get_a(x, y, c):
    n = len(x)-1
    a = np.zeros(n+1)
    for i in range(1, n):
        a[i-1] =  y[i-1]
    a[n-1] = y[n-1]
    return a

def get_b(x, y, c):
    n = len(x)-1
    b = np.zeros(n+1)
    for i in range(1, n):
        b[i-1] = ((y[i] - y[i-1])/h(i,x)) - ((h(i,x)*(c[i] + 2*c[i-1]))/3)
    b[n-1] = ( ((y[n] - y[n-1]) / h(n-1,x)) - 2*h(n-1,x)*c[n-1]/3 )
    return b

def get_d(x, y, c):
    n = len(x)-1
    d = np.zeros(n+1)
    for i in range(1, n):
        d[i-1] = (c[i] - c[i-1]) / (3*h(i,x))
    d[n-1] = (-c[n-1]/(3*h(n-1, x)))
    return d

def get_spline(X, a, b, c, d, x):
    n = len(x)
    j = 0
    for k in range(1,n):
        if ((x[k]>=X)and(x[k-1]<=X)):
            break;
        j+=1
    for i in range(n-1):
        spline = "{} + {}(x - {}) + {}(x - {})^2 + {}(x - {})^3".format(a[i], b[i], x[i], c[i], x[i], d[i], x[i])
        if (i==j):
            spline+=" V"
        print(spline)
    t = X-x[j];
    return a[j] + b[j]*t + c[j]*t*t + d[j]*t*t*t
        

In [251]:
x = [0.1,0.5,0.9,1.3,1.7]
y = [-2.2026,-0.19315,0.79464,1.5624,2.2306]
X = 0.8
c = get_c(x,y)
a = get_a(x,y,c)
b = get_b(x,y,c)
d = get_d(x,y,c)
get_spline(X, a, b, c, d, x)

-2.2026 + 5.672925892857142(x - 0.1) + 0.0(x - 0.1)^2 + -4.05813058035714(x - 0.1)^3
-0.19315 + 3.7250232142857134(x - 0.5) + -4.86975669642857(x - 0.5)^2 + 4.327215401785712(x - 0.5)^3 V
0.79464 + 1.90628125(x - 0.9) + 0.3229017857142859(x - 0.9)^2 + -0.7252622767857143(x - 0.9)^3
1.5624 + 1.8164767857142854(x - 1.3) + -0.5474129464285713(x - 1.3)^2 + 0.45617745535714266(x - 1.3)^3


0.6029136774553573